In [8]:
import pandas as pd
import os

load all data

In [9]:
stations = pd.read_csv('../Downloaded_data/2014/Stations_2014.csv')

In [10]:
rides = {}
base_path = "../Downloaded_data"

for year in range(2014, 2018):  # 2014 to 2017
    for month in range(4, 12):  # April to November 
        file_name = f"OD_{year}-{month:02d}.csv"
        file_path = os.path.join(base_path, str(year), file_name)
        
        try:
            temp_df = pd.read_csv(file_path)
            key = f"rides_{year}_{month:02d}"
            rides[key] = temp_df
            globals()[key] = temp_df  
        except FileNotFoundError:
            print(f"Missing file: {file_path} — skipped.")

all_rides = pd.concat(rides.values(), ignore_index=True)

extract only the target route  6184 ---> 6015

In [11]:
all_rides['date'] = pd.to_datetime(all_rides['start_date']).dt.date

# Group by date, route
grouped = all_rides.groupby(['date', 'start_station_code', 'end_station_code']).agg(
    duration_sec=('duration_sec', 'sum'),
    trip_count=('duration_sec', 'count'),  # count of rides (rows)
    memb_count=('is_member', 'sum')      # sum of 1s gives number of members
).reset_index()

In [12]:
grouped

,date,start_station_code,end_station_code,duration_sec,trip_count,memb_count
0,2014-04-15,5007,5006,796,1,1
1,2014-04-15,6001,6039,223,1,1
2,2014-04-15,6001,6048,786,1,0
3,2014-04-15,6001,6085,1113,1,1
4,2014-04-15,6001,6105,676,1,1
...,...,...,...,...,...,...
10735814,2017-11-15,10002,7052,1092,5,4
10735815,2017-11-15,10002,7053,206,2,2
10735816,2017-11-15,10002,7054,662,1,1
10735817,2017-11-15,10002,7055,312,1,1


In [36]:
grouped.to_csv('../Project_datasets/Aggregated_all_rides.csv')

In [13]:
grouped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10735819 entries, 0 to 10735818
Data columns (total 6 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   date                object
 1   start_station_code  int64 
 2   end_station_code    int64 
 3   duration_sec        int64 
 4   trip_count          int64 
 5   memb_count          int64 
dtypes: int64(5), object(1)
memory usage: 491.4+ MB
